In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
from google.colab import files

batch_size = 100
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 185245287.05it/s]

Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 38726244.83it/s]

Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 107271352.96it/s]

Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 19360293.46it/s]


Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



In [ ]:
class VAE(nn.Module):
    def __init__(self, x, h1, h2, z):
        super(VAE, self).__init__()

        # encoder layers
        self.e1 = nn.Linear(x, h1)
        self.e2 = nn.Linear(h1, h2)
        self.e31 = nn.Linear(h2, z)
        self.e32 = nn.Linear(h2, z)

        # decoder layers
        self.d1 = nn.Linear(z, h2)
        self.d2 = nn.Linear(h2, h1)
        self.d3 = nn.Linear(h1, x)

    # compiling encoder
    def encoder(self, _x):
        _h = F.relu(self.e1(_x))
        _h = F.relu(self.e2(_h))
        return self.e31(_h), self.e32(_h)

    # return z sample
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)

    # compiling decoder
    def decoder(self, _z):
        _h = F.relu(self.d1(_z))
        _h = F.relu(self.d2(_h))
        return F.sigmoid(self.d3(_h))

    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        _z = self.sampling(mu, log_var)
        return self.decoder(_z), mu, log_var

# build model
vae = VAE(784, 1024, 512, 2)
if torch.cuda.is_available():
    vae.cuda()

In [ ]:
optimizer = optim.Adam(vae.parameters())
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [ ]:
def train(epoch):
    vae.train()
    train_loss = 0
    for data, _ in train_loader:

        data = data.cuda()
        optimizer.zero_grad()

        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)

        loss.backward()
        train_loss += loss.item()
        optimizer.step()

    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [ ]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)

            test_loss += loss_function(recon, data, mu, log_var).item()

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
for epoch in range(1, 101):
    train(epoch)
    test()

====> Epoch: 1 Average loss: 172.1103
====> Test set loss: 156.8260
====> Epoch: 2 Average loss: 154.2797
====> Test set loss: 152.1606
====> Epoch: 3 Average loss: 150.5117
====> Test set loss: 149.9285
====> Epoch: 4 Average loss: 148.2230
====> Test set loss: 148.3414
====> Epoch: 5 Average loss: 146.5997
====> Test set loss: 147.1026
====> Epoch: 6 Average loss: 145.3655
====> Test set loss: 145.5344
====> Epoch: 7 Average loss: 144.2328
====> Test set loss: 144.9117
====> Epoch: 8 Average loss: 143.6316
====> Test set loss: 144.4136
====> Epoch: 9 Average loss: 142.7266
====> Test set loss: 143.2856
====> Epoch: 10 Average loss: 142.0439
====> Test set loss: 142.4132
====> Epoch: 11 Average loss: 141.7561
====> Test set loss: 142.7780
====> Epoch: 12 Average loss: 141.3691
====> Test set loss: 142.2707
====> Epoch: 13 Average loss: 140.8531
====> Test set loss: 141.4252
====> Epoch: 14 Average loss: 140.3999
====> Test set loss: 141.0605
====> Epoch: 15 Average loss: 140.1481
====

In [ ]:
with torch.no_grad():
    z = torch.randn(64, 2).cuda()
    sample = vae.decoder(z).cuda()

    save_image(sample.view(64, 1, 28, 28), 'sample_' + '.png')
    files.download("sample_.png")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>